<a href="https://colab.research.google.com/github/cabraldasilvac/IA_embedding/blob/main/IA_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install -q -U google-generativeai

In [18]:
# Imports the Python SDK
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
# Used to securely store your API key

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [19]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [20]:
# Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
  "\n"
  "Artigo completo:\n"
  "\n"
  "Gemini API & Foofle Ai Studio: Uma maneira ascessivel de explorar e criar protótipos com aplica¸¸ões de IA generativa")

embedding = genai.embed_content(model= "models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")
print(embedding)


{'embedding': [0.04844053, -0.035917137, -0.019293688, 0.005762169, 0.058928654, 0.006648064, -0.015751507, -0.024673862, 0.06232743, 0.07506914, 0.008469661, -0.002481703, -0.049722023, -0.023164205, 0.002805347, -0.03272419, 0.043093536, -0.001327345, -0.041826453, 0.01617021, 0.009663638, 0.0039166175, -0.026559763, -0.058783483, -0.02274478, 0.009146763, 0.023807684, -0.046119187, -0.010483876, 0.011249932, -0.04881188, 0.050467312, -0.03828967, 0.000561991, -0.03746096, -0.03365815, -0.022898281, -0.02461496, -0.010501918, -0.013538651, 0.012320907, -0.082929336, -0.0029355478, 0.025161041, 0.013018977, -0.019267678, 0.025603017, 0.060052667, 0.0037308335, -0.053653833, 0.017301597, 0.04349854, 0.061094146, -0.03661689, 0.0055791414, -0.0035065992, 0.018027157, -0.011849799, 0.02249343, 0.0031561097, -0.022674723, 0.024569917, -0.021891765, 0.04379763, 0.016895508, -0.059754744, -0.043613244, 0.008066296, 0.032446224, 0.054381933, -6.698543e-05, 0.0074607623, 0.058800202, -0.04393

In [21]:
# Listar documentos

In [22]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}

DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}

DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

**Data Frame**


In [23]:
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']
df

,Titulo,Conteudo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [24]:
model="models/embedding-001"

In [25]:
def embed_fn(title, text):
  return genai.embed_content(model= model,
                                content=text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [26]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [27]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model= model,
                                content=consulta,
                                task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [28]:
consulta = "Como faço para trocar marchas em um carro do Google?"


trecho = gerar_e_buscar_consulta(consulta, df, model)

print(trecho)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.
